In [1]:
%load_ext autoreload
%autoreload 2
import ipyleaflet as L
from dataloader import TripsLoader
from typing import Literal
from ipywidgets import Layout


In [2]:
loader = TripsLoader('05-09-2022')


In [3]:
def get_trips_by_load(load: Literal['Stone', 'Equipment', 'Soil', '4']):
    pass

In [4]:
from osgeo import gdal

filename=gdal.Open("data/DroneData_2022-03-18/P07_22.03.18_Skaret-Orthomosaic.tiff")

metadata=filename.GetMetadata()

print(metadata)


{'AREA_OR_POINT': 'Area', 'TIFFTAG_SOFTWARE': 'pix4dmapper'}


In [5]:
m = L.Map(layout=Layout(width='60%', height='700px'), center=[59.95, 10.3])
machine = loader._machines[13]
for trip in machine.trips[:100]:
    poly = L.Polyline(locations=trip.latlons)
    m.add_layer(poly)
display(m)

Map(center=[59.95, 10.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [7]:
import rasterio as rio
from pyproj import Transformer
import cv2

for i, place in enumerate(["Skaret", "Nordlandsdalen"]):
    in_path = f'data/DroneData_2022-03-18_to_2022-05-22/SINTEF - Bjørumskaret/P0{i + 7}_22.03.18_{place}-Orthomosaic.tiff'

    with rio.open(in_path) as src:
        img = src.read()
        min_lon, min_lat, max_lon, max_lat = src.bounds

    transformer = Transformer.from_crs("EPSG:5110", "EPSG:4326")
    # cv2.imwrite(f'tmp/{place}.png', cv2.cvtColor(img.transpose(1, 2, 0), cv2.COLOR_RGB2BGR))
        
    ## Conversion from UTM to WGS84 CRS
    bounds_orig = [transformer.transform(*bounds) for bounds in [[min_lat, min_lon], [max_lat, max_lon]]]
    print(bounds_orig)
    # Overlay raster (RGB) called img using add_child() function (opacity and bounding box set)
    img= L.ImageOverlay(url=f'tmp/{place}.png',
                                    bounds = bounds_orig)
    m.add_layer(img)
m

[(59.972234034659486, 10.324105050358586), (59.97856148112283, 10.334516067023081)]
[(59.9723922966532, 10.319350863592176), (59.9787739503213, 10.325758516227076)]


Map(bottom=4874735.0, center=[59.97544869310396, 10.327513217926027], controls=(ZoomControl(options=['position…